In [1]:
# import packages
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import cohen_kappa_score
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

Using TensorFlow backend.


In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
url = 'https://raw.githubusercontent.com/youko70s/NeuralNets-DeepLearning/master/training_set.tsv'
df = pd.read_csv(url, sep='\t', encoding='ISO-8859-1')

scores = df['domain1_score']
dataset = df.loc[:,['essay_id', 'essay_set', 'essay', 'domain1_score']]

In [0]:
# Generating word tokens after removing characters other than alphabets, converting them to lower case and
# removing stopwords from the text
def word_tokens(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    words = text.lower().split()
    stop_words = set(stopwords.words("english"))
    words = [w for w in words if not w in stop_words]
    return words

In [0]:
def sentence_tokens(essay_text):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_tokens = tokenizer.tokenize(essay_text.strip())
    sentences = []
    for sent_token in sent_tokens:
        if len(sent_token) > 0:
            sentences.append(word_tokens(sent_token))
    return sentences

In [0]:
# function to create feature vectors 
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

In [0]:
# function to generate word vectors 
def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay_text in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay_text, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [0]:
# build NN model 
def get_model():
    model = Sequential()
    model.add(LSTM(500, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 500], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [0]:
# set cross validation 
cv = KFold(n_splits=5, shuffle=True)
results = []
y_pred_list = []

In [54]:
print(cv)

KFold(n_splits=5, random_state=None, shuffle=True)


In [55]:
count = 1
for traincv, testcv in cv.split(dataset):
    print("###################Fold {}##################\n".format(count))
    X_test, X_train, y_test, y_train = dataset.iloc[testcv], dataset.iloc[traincv], scores.iloc[testcv], scores.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training set of essays.
            sentences += sentence_tokens(essay)
            
    # Initializing variables for word2vec model.
    num_features = 500 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_text in train_essays:
        clean_train_essays.append(word_tokens(essay_text))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_text in test_essays:
        clean_test_essays.append(word_tokens(essay_text))
    testDataVecs = getAvgFeatureVecs(clean_test_essays, model, num_features)
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    # Mean squared error
    print("Mean squared error: {0:.2f}".format(mean_squared_error(y_test.values, y_pred)))
    # Explained variance score: 1 is perfect prediction
    print('Explained Variance: {0:.2f}'.format(explained_variance_score(y_test.values, y_pred)))  
    #Cohen's kappa score
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {0:.2f}".format(result))
    results.append(result)

    count += 1

###################Fold 1##################

Training Word2Vec Model...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 1, 500)            2002000   
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                144640    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 2,146,705
Trainable params: 2,146,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10380/10380 [==============================] - 15s 1ms/step - loss: 61.5176 - mean_absolute_error: 4.1766
Epoch 2/50
10380/10380 [==============================] - 12s 1ms/step - loss: 37.8270 - mean_absolute_error: 3.4079
Epoch 3/

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 1, 500)            2002000   
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                144640    
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 2,146,705
Trainable params: 2,146,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10381/10381 [==============================] - 15s 1ms/step - loss: 57.2655 - mean_absolute_error: 4.0529
Epoch 2/50
10381/10381 [==============================] - 12s 1ms/step - loss: 36.0316 - mean_absolute_error: 3.3893
Epoch 3/

...

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 1, 500)            2002000   
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                144640    
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 2,146,705
Trainable params: 2,146,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10381/10381 [==============================] - 16s 2ms/step - loss: 58.6125 - mean_absolute_error: 4.0624
Epoch 2/50
10381/10381 [==============================] - 13s 1ms/step - loss: 36.5700 - mean_absolute_error: 3.3696
Epoch 3/

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 1, 500)            2002000   
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                144640    
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 2,146,705
Trainable params: 2,146,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10381/10381 [==============================] - 16s 2ms/step - loss: 57.9336 - mean_absolute_error: 4.0653
Epoch 2/50
10381/10381 [==============================] - 12s 1ms/step - loss: 36.6238 - mean_absolute_error: 3.4140
Epoch 3/

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 1, 500)            2002000   
_________________________________________________________________
lstm_16 (LSTM)               (None, 64)                144640    
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 2,146,705
Trainable params: 2,146,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10381/10381 [==============================] - 17s 2ms/step - loss: 59.0207 - mean_absolute_error: 4.1179
Epoch 2/50
10381/10381 [==============================] - 13s 1ms/step - loss: 37.1447 - mean_absolute_error: 3.4363
Epoch 3/